In [ ]:
import pandas as pd
import numpy as np
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving completeData.csv to completeData.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['completeData.csv']))

In [ ]:
df = pd.read_csv('completeData.csv')
df.head()

In [ ]:
data = pd.DataFrame(df,columns = ['subject','hamspam'])
data.head()

,subject,hamspam
0,Blow out Sale on printer ink. Save up to 85%,Spam
1,CONGRATS! You have Scored 85% Special Discount...,Spam
2,CONGRATS! You have Scored 85% Special Discount...,Spam
3,Printer ink sale up to 85% off plus free shipp...,Spam
4,Printer Ink- Save up to 85% off,Spam


In [ ]:
from string import printable
st = set(printable)
data["hamspam"]=data["hamspam"].apply(lambda x: 1 if x.lower() == "spam" else 0)
data["subject"] = data["subject"].apply(lambda x: ''.join(["" if  i not in st else i for i in x]))
data.head()

,subject,hamspam
0,Blow out Sale on printer ink. Save up to 85%,1
1,CONGRATS! You have Scored 85% Special Discount...,1
2,CONGRATS! You have Scored 85% Special Discount...,1
3,Printer ink sale up to 85% off plus free shipp...,1
4,Printer Ink- Save up to 85% off,1


In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Remove stopwords from the data

In [ ]:
import re
import string
from string import punctuation
from nltk.corpus import stopwords
def text_processing(message):
    Stopwords = stopwords.words('english')
    no_punctuation = [char for char in message if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)  
    return ' '.join([word for word in no_punctuation.split() if word.lower() not in Stopwords])
data['subject'] = data['subject'].apply(text_processing)
data.head()

,subject,hamspam
0,Blow Sale printer ink Save 85,1
1,CONGRATS Scored 85 Special Discount Ink Toner,1
2,CONGRATS Scored 85 Special Discount Ink Toner,1
3,Printer ink sale 85 plus free shipping option,1
4,Printer Ink Save 85,1


In [ ]:
from sklearn.model_selection import train_test_split
X=data['subject'].values
y=data['hamspam'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Count Vectorize and TF-IDF transform

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#Vectorization
vect = CountVectorizer()
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)
#Term Frequency, Inverse Document Frequency
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
X_train=X_train.toarray()
X_test=X_test.toarray()

Fit model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
model = Sequential()
model.add(Dense(units=3,activation='relu'))
#model.add(Dense(units=32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=2,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(loss='binary_crossentropy', optimizer='adam')
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model.fit(x=X_train,y=y_train,epochs=40,validation_data=(X_test, y_test), verbose=1,callbacks=[early_stop])

Epoch 1/40
484/484 [==============================] - 3s 3ms/step - loss: 0.6038 - val_loss: 0.2633
Epoch 2/40
484/484 [==============================] - 1s 2ms/step - loss: 0.2152 - val_loss: 0.1558
Epoch 3/40
484/484 [==============================] - 1s 2ms/step - loss: 0.1394 - val_loss: 0.1534
Epoch 4/40
484/484 [==============================] - 1s 2ms/step - loss: 0.1024 - val_loss: 0.1595
Epoch 5/40
484/484 [==============================] - 1s 2ms/step - loss: 0.0865 - val_loss: 0.1998
Epoch 6/40
484/484 [==============================] - 1s 2ms/step - loss: 0.0808 - val_loss: 0.2399
Epoch 7/40
484/484 [==============================] - 1s 2ms/step - loss: 0.0742 - val_loss: 0.2411
Epoch 8/40
484/484 [==============================] - 1s 2ms/step - loss: 0.0674 - val_loss: 0.2599
Epoch 9/40
484/484 [==============================] - 1s 2ms/step - loss: 0.0635 - val_loss: 0.2777
Epoch 10/40
484/484 [==============================] - 1s 2ms/step - loss: 0.0607 - val_loss: 0.2878

In [ ]:
print(len(X_test))

3873


In [ ]:
predictions = model.predict_classes(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[2292  119]
 [  63 1399]]
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      2411
           1       0.92      0.96      0.94      1462

    accuracy                           0.95      3873
   macro avg       0.95      0.95      0.95      3873
weighted avg       0.95      0.95      0.95      3873



In [ ]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96      2411
           1       0.92      0.96      0.94      1462

    accuracy                           0.95      3873
   macro avg       0.95      0.95      0.95      3873
weighted avg       0.95      0.95      0.95      3873

